# When Explanations Lie: Why Many Modified BP Attributions fails


## Two classes: Dog and cat saliency map for figure 1.



In [ ]:
# select gpu
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow
import tensorflow as tf

import innvestigate
import matplotlib.pyplot as plt

import numpy as np
import PIL 
import copy
import contextlib

import imp
import numpy as np
import os

from skimage.measure import compare_ssim 
import pickle
from collections import OrderedDict
from IPython.display import IFrame, display

import keras
import keras.backend
import keras.models

from keras.applications.resnet50 import preprocess_input

import innvestigate
import innvestigate.applications.imagenet
import innvestigate.utils as iutils
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.analyzer.relevance_based.relevance_analyzer import LRP
from innvestigate.analyzer.base import AnalyzerNetworkBase, ReverseAnalyzerBase
from innvestigate.analyzer.deeptaylor import DeepTaylor

import time
import tqdm

import seaborn as sns

import itertools
import matplotlib as mpl

from tensorflow.python.client import device_lib
from scipy.special import softmax

import sys

from when_explanations_lie import *
import json

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)
device_lib.list_local_devices()

In [ ]:
# set the path to imagenet in `imagenet_dir.json`
host, = ! hostname

with open('imagenet_dir.json') as f:
    imagenet_dir = json.load(f)[host]
    
print("using imagenet at: ", imagenet_dir)

# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
n_selected_imgs = 200

model_names = ['resnet50', 'vgg16']

os.makedirs('figures', exist_ok=True)
# https://pixabay.com/photos/cat-kitten-dog-puppy-pet-2603395/

In [ ]:
def load_model_and_meta(model_name, load_weights=True, clear_session=True):
    if clear_session:
        keras.backend.clear_session()
    if model_name in ['vgg16', 'resnet50']:
        model, innv_net, color_conversion = load_model(model_name, load_weights) 
        meta = ImageNetMeta(model, model_name, innv_net, n_selected_imgs, 
                            imagenet_dir, ex_image_path)
    elif model_name == 'cifar10':
        model, _, _ = load_model('cifar10', load_weights)
        meta = CIFAR10Meta(model, n_selected_imgs)
    else:
        raise ValueError()
    return model, meta

In [ ]:
model, meta = load_model_and_meta('vgg16')

In [ ]:
correct = []
for img, target in tqdm.tqdm_notebook(meta.images):
    logits = model.predict(img)
    top1 = logits.argmax()
    correct.append(top1 == target)
    
print("Top-1 accuracy: ", np.mean(correct))

In [ ]:
with open('imagenet_labels.json', 'r') as f:
    idx_to_label =  json.load(f)

In [ ]:
#! cp -r ~/images .

In [ ]:
cat_dog = PIL.Image.open('images/cat_dog_224.jpg')
cat  = PIL.Image.open('images/cat_224.jpg')
dog  = PIL.Image.open('images/dog_224.jpg')

zebra_elephant = PIL.Image.open('images/zebra_elephant_224.jpg')
zebra  = PIL.Image.open('images/zebra_224.jpg')
elephant  = PIL.Image.open('images/elephant_224.jpg')

two_class_pil_imgs = [
    # Persian cat, King Charles Spaniel 
    ('cat_dog', cat_dog, cat, dog, 283, 156),  
    # Zebra, African bush elephant
    ('zebra_elephant', zebra_elephant, zebra, elephant, 340, 386)
]
two_class_imgs = [(
    name,
    preprocess_input(np.array(a_and_b).astype(np.float))[None], 
    preprocess_input(np.array(a).astype(np.float))[None], 
    preprocess_input(np.array(b, ).astype(np.float))[None], 
    a_cls,
    b_cls
) for name, a_and_b, a, b, a_cls, b_cls in two_class_pil_imgs]

In [ ]:
for img_name, ab, a, b, a_cls, b_cls in two_class_imgs:
    logit_ab = model.predict(ab)[0]
    logit_a = model.predict(a)[0]
    logit_b = model.predict(b)[0]
    
    idx_ab = np.argsort(logit_ab)[::-1]
    idx_a = np.argsort(logit_a)[::-1]
    idx_b = np.argsort(logit_b)[::-1]
    
    
    prob_a = softmax(logit_a, -1)
    prob_b = softmax(logit_b, -1)
    
    for top_class in idx_a[:5]:
        print(prob_a[top_class], top_class, idx_to_label[int(top_class)])
        #print()
        
    print()
        
    for top_class in idx_b[:5]:
        print(prob_b[top_class], top_class, idx_to_label[int(top_class)])
        
    print()
    print("-" * 60)
    print()

In [ ]:
def normalize_neg(x):
    vmax = np.percentile(x, 99)
    vmin = np.percentile(x, 1)
    x_pos = x * (x > 0)
    x_neg = x * (x < 0)
    x_pos = x_pos / vmax
    x_neg = - x_neg / vmin
    return np.abs(np.clip(x_pos + x_neg, -1, 1))


In [ ]:
innvestigate.analyzer.analyzers

In [ ]:
from monkey_patch_lrp_resnet import custom_add_bn_rule, get_custom_rule

In [ ]:
meta.model_name

In [ ]:
model

In [ ]:
hmaps = OrderedDict()
for model_name in ['vgg16']:
    print(model_name)
    #keras.backend.clear_session()
    #model, meta = load_model_and_meta(model_name)
    
    with custom_add_bn_rule(get_custom_rule('lrp.alpha_1_beta_0', {})):
        analyser = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model, neuron_selection_mode="index")
        analyser.create_analyzer_model()
    
        for (img_name, ab, a, b, a_cls, b_cls) in two_class_imgs:
            ab_explain_a = analyser.analyze(ab, a_cls)
            ab_explain_b = analyser.analyze(ab, b_cls)

            a_explain_a = analyser.analyze(a, a_cls)
            b_explain_b = analyser.analyze(b, b_cls)

            random_logit = np.random.choice([i for i in range(1000) if i not in [a_cls, b_cls]])
            ab_explain_random = analyser.analyze(ab, random_logit)

            hmaps[model_name, img_name, 'ab_explain_a'] = ab_explain_a
            hmaps[model_name, img_name, 'ab_explain_b'] = ab_explain_b
            hmaps[model_name, img_name, 'ab_explain_random'] = ab_explain_random
            hmaps[model_name, img_name, 'a_explain_a'] = a_explain_a
            hmaps[model_name, img_name, 'b_explain_b'] = b_explain_b


In [ ]:
def normalize_abs(x, percentile=99.5):
    return np.abs(normalize_visual(x, percentile))

In [ ]:
figdir = 'figures/two_class/'
! rm {figdir}/*
! mkdir -p {figdir}

In [ ]:
def plot_hmap(hmap, save_path):
    plt.figure(figsize=(2, 2))
    plt.imshow(normalize_abs(hmap[0].sum(-1), 99.5), 
               vmin=-1, vmax=1, cmap='seismic') 
    plt.xticks([])
    plt.yticks([])
    if save_path is not None:
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0)
    plt.show()
    
    
    
    
def plot_hmap_diff(hmap, save_path):
    plt.figure(figsize=(3, 2))
    hmap = hmap[0].sum(-1) 
    hmap = np.clip(hmap, np.percentile(hmap, 1.0), np.percentile(hmap, 99.0))
    abs_max = max(np.abs(hmap.min()), np.abs(hmap.max()))
    vmin = - abs_max
    vmax = abs_max
    
    plt.imshow(hmap, vmin=vmin, vmax=vmax, cmap='seismic') 
    cbar = plt.colorbar()
    cbar.ax.tick_params(labelsize=14)
    plt.xticks([])
    plt.yticks([])
    if save_path is not None:
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0)
    plt.show()

def plot_image(img, save_path=None):
    plt.figure(figsize=(2, 2))
    plt.imshow(image(img[0])[:, :, [2,1,0]])
    plt.xticks([])
    plt.yticks([])
    if save_path is not None:
        plt.tight_layout()
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.0)
    plt.show()

def get_img_path(img_name, class_name):
    return os.path.join(figdir, "image_{}_{}.pdf".format(img_name, class_name))

def get_hmap_path(img_name, img_cls, explained_cls):
    return os.path.join(figdir, "hmap_{}_{}_{}.pdf".format(img_name, img_cls, explained_cls))

def get_l1_dist_path(img_name):
    return os.path.join(figdir, "l1_dist_{}.txt".format(img_name))

with sns.axes_style('white', {'font.family': 'serif'}):
    for model_name in ['vgg16']:
        for (img_name, ab, a, b, a_cls, b_cls) in two_class_imgs:
            cls_a = idx_to_label[a_cls].replace(' ', '_')
            cls_b = idx_to_label[b_cls].replace(' ', '_')

            print(cls_a, a_cls, cls_b, b_cls)
            plot_image(ab, get_img_path(img_name, 'both'))
            plot_image(a, get_img_path(img_name, cls_a))
            plot_image(b, get_img_path(img_name, cls_b))

            hmap  = hmaps[model_name, img_name, 'a_explain_a']
            plot_hmap(hmap, get_hmap_path(img_name, cls_a, cls_a))

            hmap  = hmaps[model_name, img_name, 'b_explain_b']
            plot_hmap(hmap, get_hmap_path(img_name, cls_b, cls_b))

            hmap  = hmaps[model_name, img_name, 'ab_explain_a']
            plot_hmap(hmap, get_hmap_path(img_name, 'both', cls_a))


            hmap  = hmaps[model_name, img_name, 'ab_explain_b']
            plot_hmap(hmap, get_hmap_path(img_name, 'both', cls_b))

            hmap  = hmaps[model_name, img_name, 'ab_explain_random']
            plot_hmap(hmap, get_hmap_path(img_name, 'both', 'random'))

            hmap_a  = hmaps[model_name, img_name, 'ab_explain_a']
            hmap_b  = hmaps[model_name, img_name, 'ab_explain_b']
            hmap_a = normalize_abs(hmap_a)
            hmap_b = normalize_abs(hmap_b)
            plot_hmap_diff(hmap_a - hmap_b, get_hmap_path(img_name, 'both', 'diff'))

            l1_distance = np.abs(normalize_abs(hmaps[model_name, img_name, 'ab_explain_a']) -  
                                 normalize_abs(hmaps[model_name, img_name, 'ab_explain_b'])).mean()
            with open(get_l1_dist_path(img_name), 'w') as f:
                f.write("{:.6f}".format(l1_distance))

            print("L1 distance: {:.6f}".format(l1_distance))

In [ ]:
display(IFrame(figdir + '/hmap_cat_dog_both_King_Charles_Spaniel.pdf', 800, 600))

In [ ]:
! ls {figdir}